# 1. Set up environment

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!apt update

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [973 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,402 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-u

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.osuosl.org/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar xf spark-3.5.1-bin-hadoop3.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"

In [ ]:
import findspark
findspark.init()
findspark.find()

'/content/spark-3.5.1-bin-hadoop3'

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .appName("MAPPING")\
        .getOrCreate()

# 2. Read files

## 2.1. Vote table

In [ ]:
data_gz2 = spark.read.csv('/content/drive/MyDrive/University/Academic/IU/SEM 6/Big Data Analytics for Remote Sensing Laboratory/Final project/gz2_hart16.csv', header=True)

In [ ]:
data_gz2.show(5)

+------------------+------------------+------------------+-----------+-----------+--------+---------+---------------------+-----------+---------------------------------------+----------------------------------------+------------------------------------------+---------------------------------------------------+------------------------------------------+--------------------------------------+-------------------------------------------------+--------------------------------------------------+----------------------------------------------------+-------------------------------------------------------------+----------------------------------------------------+------------------------------------------------+-------------------------------------------------+--------------------------------------------------+----------------------------------------------------+-------------------------------------------------------------+----------------------------------------------------+--------------------

In [ ]:
data_gz2.printSchema()

root
 |-- dr7objid: string (nullable = true)
 |-- ra: string (nullable = true)
 |-- dec: string (nullable = true)
 |-- rastring: string (nullable = true)
 |-- decstring: string (nullable = true)
 |-- sample: string (nullable = true)
 |-- gz2_class: string (nullable = true)
 |-- total_classifications: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- t01_smooth_or_features_a01_smooth_count: string (nullable = true)
 |-- t01_smooth_or_features_a01_smooth_weight: string (nullable = true)
 |-- t01_smooth_or_features_a01_smooth_fraction: string (nullable = true)
 |-- t01_smooth_or_features_a01_smooth_weighted_fraction: string (nullable = true)
 |-- t01_smooth_or_features_a01_smooth_debiased: string (nullable = true)
 |-- t01_smooth_or_features_a01_smooth_flag: string (nullable = true)
 |-- t01_smooth_or_features_a02_features_or_disk_count: string (nullable = true)
 |-- t01_smooth_or_features_a02_features_or_disk_weight: string (nullable = true)
 |-- t01_smooth_or_feat

In [ ]:
data_gz2.count()

239695

## 2.2. Mapping table

In [ ]:
map_gz2 = spark.read.csv('/content/drive/MyDrive/University/Academic/IU/SEM 6/Big Data Analytics for Remote Sensing Laboratory/gz2_filename_mapping.csv', header=True)

In [ ]:
map_gz2.show(5)

+------------------+--------+--------+
|             objid|  sample|asset_id|
+------------------+--------+--------+
|587722981736120347|original|       1|
|587722981736579107|original|       2|
|587722981741363294|original|       3|
|587722981741363323|original|       4|
|587722981741559888|original|       5|
+------------------+--------+--------+
only showing top 5 rows



In [ ]:
map_gz2.printSchema()

root
 |-- objid: string (nullable = true)
 |-- sample: string (nullable = true)
 |-- asset_id: string (nullable = true)



In [ ]:
map_gz2.count()

355990

# 3. Preprocessing

## 3.1. Join 2 table

In [ ]:
# Join the 2 table based on the 'objid' of table map_gz2 and 'dr7objid' of table data_gz2
joined_gz2 = data_gz2.join(map_gz2, data_gz2.dr7objid == map_gz2.objid, "inner")

In [ ]:
joined_gz2.count()

239695

In [ ]:
joined_gz2.show(5)

+------------------+------------------+-------------------+-----------+-----------+--------+---------+---------------------+-----------+---------------------------------------+----------------------------------------+------------------------------------------+---------------------------------------------------+------------------------------------------+--------------------------------------+-------------------------------------------------+--------------------------------------------------+----------------------------------------------------+-------------------------------------------------------------+----------------------------------------------------+------------------------------------------------+-------------------------------------------------+--------------------------------------------------+----------------------------------------------------+-------------------------------------------------------------+----------------------------------------------------+-------------------

In [ ]:
joined_gz2.printSchema()

root
 |-- dr7objid: string (nullable = true)
 |-- ra: string (nullable = true)
 |-- dec: string (nullable = true)
 |-- rastring: string (nullable = true)
 |-- decstring: string (nullable = true)
 |-- sample: string (nullable = true)
 |-- gz2_class: string (nullable = true)
 |-- total_classifications: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- t01_smooth_or_features_a01_smooth_count: string (nullable = true)
 |-- t01_smooth_or_features_a01_smooth_weight: string (nullable = true)
 |-- t01_smooth_or_features_a01_smooth_fraction: string (nullable = true)
 |-- t01_smooth_or_features_a01_smooth_weighted_fraction: string (nullable = true)
 |-- t01_smooth_or_features_a01_smooth_debiased: string (nullable = true)
 |-- t01_smooth_or_features_a01_smooth_flag: string (nullable = true)
 |-- t01_smooth_or_features_a02_features_or_disk_count: string (nullable = true)
 |-- t01_smooth_or_features_a02_features_or_disk_weight: string (nullable = true)
 |-- t01_smooth_or_feat

In [ ]:
joined_gz2 = joined_gz2.drop('objid')

In [ ]:
joined_gz2.printSchema()

root
 |-- dr7objid: string (nullable = true)
 |-- ra: string (nullable = true)
 |-- dec: string (nullable = true)
 |-- rastring: string (nullable = true)
 |-- decstring: string (nullable = true)
 |-- sample: string (nullable = true)
 |-- gz2_class: string (nullable = true)
 |-- total_classifications: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- t01_smooth_or_features_a01_smooth_count: string (nullable = true)
 |-- t01_smooth_or_features_a01_smooth_weight: string (nullable = true)
 |-- t01_smooth_or_features_a01_smooth_fraction: string (nullable = true)
 |-- t01_smooth_or_features_a01_smooth_weighted_fraction: string (nullable = true)
 |-- t01_smooth_or_features_a01_smooth_debiased: string (nullable = true)
 |-- t01_smooth_or_features_a01_smooth_flag: string (nullable = true)
 |-- t01_smooth_or_features_a02_features_or_disk_count: string (nullable = true)
 |-- t01_smooth_or_features_a02_features_or_disk_weight: string (nullable = true)
 |-- t01_smooth_or_feat

## 3.2. Check category of class

In [ ]:
from pyspark.sql import functions as F

In [ ]:
joined_gz2.select(F.col('gz2_class')).distinct().count()

818

In [ ]:
joined_gz2.groupBy('gz2_class').count().sort('count', ascending=False).show(10)

+---------+-----+
|gz2_class|count|
+---------+-----+
|       Ei|44038|
|       Er|36764|
|      Ser|14009|
|     Sc?t|13509|
|       Ec|10149|
|       Sb| 6932|
|    SBc2m| 5862|
|       Sc| 5776|
|     Sb?t| 5431|
|      Sen| 4450|
+---------+-----+
only showing top 10 rows



###Check type E (Elliptical)

In [ ]:
joined_gz2.filter(joined_gz2.gz2_class.like('E%'))\
  .groupBy('gz2_class')\
  .count()\
  .sort('count', ascending=False)\
  .show(5)

+---------+-----+
|gz2_class|count|
+---------+-----+
|       Ei|44038|
|       Er|36764|
|       Ec|10149|
|    Er(o)| 1464|
|    Ei(o)| 1051|
+---------+-----+
only showing top 5 rows



###Check type Barred spirals

In [ ]:
joined_gz2.filter(joined_gz2.gz2_class.like('SBc2l'))\
  .groupBy('gz2_class')\
  .count()\
  .sort('count', ascending=False)\
  .show(10)

+---------+-----+
|gz2_class|count|
+---------+-----+
|    SBc2l| 2017|
+---------+-----+



### Check type Spirals

In [ ]:
joined_gz2.filter(joined_gz2.gz2_class.like('Sc2t') | joined_gz2.gz2_class.like('Sc3t'))\
  .groupBy('gz2_class')\
  .count()\
  .sort('count', ascending=False)\
  .show(10)

+---------+-----+
|gz2_class|count|
+---------+-----+
|     Sc2t| 2942|
|     Sc3t| 1960|
+---------+-----+



### Check type Edge on

In [ ]:
joined_gz2.filter(joined_gz2.gz2_class.like('Ser'))\
  .groupBy('gz2_class')\
  .count()\
  .sort('count', ascending=False)\
  .show(10)

+---------+-----+
|gz2_class|count|
+---------+-----+
|      Ser|14009|
+---------+-----+



### Check type Merger

In [ ]:
joined_gz2.filter(joined_gz2.gz2_class.like('Sb(m)') | joined_gz2.gz2_class.like('Sc2t(m)'))\
  .groupBy('gz2_class')\
  .count()\
  .sort('count', ascending=False)\
  .show(10)

+---------+-----+
|gz2_class|count|
+---------+-----+
|    Sb(m)|  966|
|  Sc2t(m)|   66|
+---------+-----+



## 3.3. Filter data to keep only some classes

In [ ]:
# Keep some classes
subdata_gz2 = joined_gz2.filter(joined_gz2.gz2_class.isin(['Ei', 'Er',\
                                                           'SBc2l',\
                                                           'Sc2t', 'Sc3t',\
                                                           'Ser',\
                                                           'Sb(m)', 'Sc2t(m)']))

In [ ]:
subdata_gz2.select(F.col('gz2_class')).distinct().show()

+---------+
|gz2_class|
+---------+
|      Ser|
|    SBc2l|
|     Sc2t|
|       Er|
|    Sb(m)|
|       Ei|
|  Sc2t(m)|
|     Sc3t|
+---------+



In [ ]:
subdata_gz2.count()

102762

In [ ]:
subdata_gz2\
  .groupBy('gz2_class')\
  .count()\
  .show()

+---------+-----+
|gz2_class|count|
+---------+-----+
|      Ser|14009|
|    SBc2l| 2017|
|     Sc2t| 2942|
|       Er|36764|
|    Sb(m)|  966|
|       Ei|44038|
|  Sc2t(m)|   66|
|     Sc3t| 1960|
+---------+-----+



### Some manual double checking approaches (not necessary)

In [ ]:
# Show some example of a specific class to double check with image in data
joined_gz2.filter(joined_gz2.gz2_class == 'Sc2t(m)').select('asset_id', 'dr7objid', 'gz2_class').show(5)

+--------+------------------+---------+
|asset_id|          dr7objid|gz2_class|
+--------+------------------+---------+
|    9112|587725550676672740|  Sc2t(m)|
|   11333|587725590382903534|  Sc2t(m)|
|   12509|587725816959664268|  Sc2t(m)|
|   20841|587726101483815017|  Sc2t(m)|
|   30154|587729157895356551|  Sc2t(m)|
+--------+------------------+---------+
only showing top 5 rows



In [ ]:
# Show a galaxy class based on its image ID
joined_gz2.filter(joined_gz2.asset_id == 100171).select('asset_id', 'dr7objid', 'gz2_class').show(5)

+--------+------------------+---------+
|asset_id|          dr7objid|gz2_class|
+--------+------------------+---------+
|  100171|587736915147817023|     Sc2t|
+--------+------------------+---------+



## 3.4. New table with 1000 galaxies

In [ ]:
from pyspark.sql.functions import rand

In [ ]:
classes = ['Ei', 'Er', 'SBc2l', 'Sc2t', 'Sc3t', 'Ser', 'Sc2t(m)', 'Sb(m)']
# Initiate new df
mini_gz2 = spark.createDataFrame([], subdata_gz2.schema)
# Filter the subdata
for c in classes:
  if c == 'SBc2l' or c == 'Ser':
    num_sam = 200
  else:
    if c == 'Sb(m)':
      num_sam = 134     ## Sc2t(m) < 100
    else:
      num_sam = 100
  mini = subdata_gz2.filter(subdata_gz2.gz2_class == c).orderBy(rand()).limit(num_sam)
  mini_gz2 = mini_gz2.union(mini)

In [ ]:
mini_gz2.groupBy('gz2_class').count().show()

+---------+-----+
|gz2_class|count|
+---------+-----+
|       Ei|  100|
|       Er|  100|
|    SBc2l|  200|
|     Sc2t|  100|
|     Sc3t|  100|
|      Ser|  200|
|  Sc2t(m)|   66|
|    Sb(m)|  134|
+---------+-----+



In [ ]:
mini_gz2.count()

1000

In [ ]:
mini_gz2.show(5)

+------------------+------------------+------------------+-----------+-----------+--------+---------+---------------------+-----------+---------------------------------------+----------------------------------------+------------------------------------------+---------------------------------------------------+------------------------------------------+--------------------------------------+-------------------------------------------------+--------------------------------------------------+----------------------------------------------------+-------------------------------------------------------------+----------------------------------------------------+------------------------------------------------+-------------------------------------------------+--------------------------------------------------+----------------------------------------------------+-------------------------------------------------------------+----------------------------------------------------+--------------------

## 3.5. Combine 2 sub-merger classes to 1 class Merger

In [ ]:
mini7_gz2 = mini_gz2.withColumn('gz2_class', F.when(mini_gz2.gz2_class == 'Sc2t(m)', 'm')\
                                            .when(mini_gz2.gz2_class == 'Sb(m)', 'm')\
                                            .otherwise(mini_gz2.gz2_class))

In [ ]:
mini7_gz2.groupBy('gz2_class').count().show()

+---------+-----+
|gz2_class|count|
+---------+-----+
|       Ei|  100|
|       Er|  100|
|    SBc2l|  200|
|     Sc2t|  100|
|     Sc3t|  100|
|      Ser|  200|
|        m|  200|
+---------+-----+



# 4. Write output to disk

In [ ]:
mini7_gz2.count()

1000

In [ ]:
mini7_gz2.show(5)

+------------------+------------------+------------------+-----------+-----------+--------+---------+---------------------+-----------+---------------------------------------+----------------------------------------+------------------------------------------+---------------------------------------------------+------------------------------------------+--------------------------------------+-------------------------------------------------+--------------------------------------------------+----------------------------------------------------+-------------------------------------------------------------+----------------------------------------------------+------------------------------------------------+-------------------------------------------------+--------------------------------------------------+----------------------------------------------------+-------------------------------------------------------------+----------------------------------------------------+--------------------

In [ ]:
# Drop duplicate column from joining 2 tables
mini7_gz2 = mini7_gz2.drop('sample')

In [ ]:
mini7_gz2.printSchema()

root
 |-- dr7objid: string (nullable = true)
 |-- ra: string (nullable = true)
 |-- dec: string (nullable = true)
 |-- rastring: string (nullable = true)
 |-- decstring: string (nullable = true)
 |-- gz2_class: string (nullable = true)
 |-- total_classifications: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- t01_smooth_or_features_a01_smooth_count: string (nullable = true)
 |-- t01_smooth_or_features_a01_smooth_weight: string (nullable = true)
 |-- t01_smooth_or_features_a01_smooth_fraction: string (nullable = true)
 |-- t01_smooth_or_features_a01_smooth_weighted_fraction: string (nullable = true)
 |-- t01_smooth_or_features_a01_smooth_debiased: string (nullable = true)
 |-- t01_smooth_or_features_a01_smooth_flag: string (nullable = true)
 |-- t01_smooth_or_features_a02_features_or_disk_count: string (nullable = true)
 |-- t01_smooth_or_features_a02_features_or_disk_weight: string (nullable = true)
 |-- t01_smooth_or_features_a02_features_or_disk_fraction: st

### Convert to panda to save csv

In [ ]:
pd_mini7_gz2 = mini7_gz2.toPandas()

               dr7objid                  ra                 dec     rastring  \
0    588010879833997510   182.7119903564453   5.422131538391113  12:10:50.88   
1    587738946673639494  155.95065307617188  35.693973541259766  10:23:48.16   
2    588017703489241485  230.74630737304688   8.664599418640137  15:22:59.11   
3    587728880341876898   157.7476043701172   4.589554786682129  10:30:59.42   
4    587733079742283987   196.3650665283203  53.699623107910156  13:05:27.62   
..                  ...                 ...                 ...          ...   
995  588011102643945637  239.99334716796875   48.34272003173828  15:59:58.40   
996  588007004179333224  196.91868591308594   63.91506576538086  13:07:40.48   
997  587735349087044015  126.86070251464844   8.619308471679688  08:27:26.57   
998  588018253227425833   251.5872802734375   26.83464813232422  16:46:20.95   
999  587726032262791365  212.60780334472656  1.9818413257598877  14:10:25.87   

       decstring gz2_class total_classi

In [ ]:
pd_mini7_gz2.to_csv('/content/drive/MyDrive/University/Academic/IU/SEM 6/Big Data Analytics for Remote Sensing Laboratory/Final project/mini_1000s_7c_gz2_hart16_mapping.csv',\
                    header=True, index=False)